In [1]:
import os
from ecdsa import SigningKey, VerifyingKey, SECP256k1
from ecdsa.util import randrange_from_seed__trytryagain
from hashlib import sha256
import time
import calendar

In [2]:
class Wallet:
    """
    A class that represents an individual account.
    
    Attributes
    ----------
    seed : length 32 hexadecimal string
        represents 16 bytes, used in deterministically generating key-pair
    sexexp : int
        secret exponent; used in determining ecdsa key pair; generated from seed
    private_key: ecdsa.keys.SigningKey
        generated from secret exponent; used to sign messages
    public_key: ecdsa.keys.VerifyingKey
        generated from private_key; used to verify digital signatures sined by private_key
    """
    
    def __init__(self, seed_hex=None, display=False):
        """
        initializes wallet
        
        Parameters
        ----------
        seed_hex : length 32 hexadecimal string, optional
            represents 16 bytes, used in deterministically generating key-pair (default is random hex value)
        """
        if seed_hex:
            self.seed = bytes.fromhex(seed_hex)
        else:
            self.seed = os.urandom(SECP256k1.baselen)
            
        self.secexp = randrange_from_seed__trytryagain(self.seed, SECP256k1.order)
        self.private_key = SigningKey.from_secret_exponent(self.secexp, curve=SECP256k1)
        self.public_key = self.private_key.verifying_key
        self.display_creation(display)
        
    def sign_message(self, message, display=False):
        """
        digitally sign a message using ecdsa (SECP256k1)
        
        Parameters
        ----------
        message : string
            any arbitrary string to digitally sign
        """
        encoded_message = message if isinstance(message, bytes) else message.encode()
        
        signature = self.private_key.sign_deterministic(encoded_message)
        if display:
            print(f'signature: {signature.hex()}')
        return signature
    
    def verify_signature(self, signature, message, display=False):
        """
        verify a given digital signature against the message it represents
        
        Parameters
        ----------
        signature : bytes
            byte encoded digital signature, signed by private_key
        message : string
            arbitrary string that the signature is representing
        """
        encoded_message = message if isinstance(message, bytes) else message.encode()
        
        is_valid = self.public_key.verify(signature, encoded_message)
        if display:
            print(f'message: {message}, actual sig: {self.sign_message(message, display).hex()}, given sig: {signature.hex()}')
        return is_valid
    
    def display_creation(self, display):
        if display:
            print( f'Wallet seed: {self.seed.hex()}' )
            print( f'Wallet secret exponent: {self.secexp}')
            print( f'Wallet private key: {self.private_key.to_string().hex()}' )
            print( f'Wallet public key: {self.public_key.to_string().hex()}' )

In [3]:
test_wallet = Wallet(seed_hex='6daa908eb3d309183a9f4789a00f51b18542da1417be0cdd799081228236dae9', display=True)

Wallet seed: 6daa908eb3d309183a9f4789a00f51b18542da1417be0cdd799081228236dae9
Wallet secret exponent: 85314299752260761361628249051057399690231672481648748497211137228769388769381
Wallet private key: bc9e2eb5d3a6857f52803f57750a2e7c9a75e8d1a14784e61f38845ecc6db065
Wallet public key: 57b28b2862c10eb35a0cc9799543ad75ed88c1151396993e5d43bb2f1851acd2665e92bff0daf899374c1bf6723b84d112bdfed23ace5fa2f00e629080f7d638


In [4]:
test_sig = test_wallet.sign_message('hello world', display=True)

signature: 9bfced9bf3c362c1620fea4450435aa28b352fd7fb15392ca289cf1c0f9b0d9ecaf2ca82bff7fdccb44e4876870efa3dfde78eb900b6e2229da93433b326b700


In [5]:
bad_sig = bytes.fromhex('9bfced9bf3c362c1620fea4450435aa28b352fd7fb15392ca289cf1c0f9b0d9ecaf2ca82bff7fdccb44e4876870efa3dfde78eb900b6e2229da93433b326b701')

In [6]:
test_verify_fail = test_wallet.verify_signature(test_sig, 'hello world', display=True)

signature: 9bfced9bf3c362c1620fea4450435aa28b352fd7fb15392ca289cf1c0f9b0d9ecaf2ca82bff7fdccb44e4876870efa3dfde78eb900b6e2229da93433b326b700
message: hello world, actual sig: 9bfced9bf3c362c1620fea4450435aa28b352fd7fb15392ca289cf1c0f9b0d9ecaf2ca82bff7fdccb44e4876870efa3dfde78eb900b6e2229da93433b326b700, given sig: 9bfced9bf3c362c1620fea4450435aa28b352fd7fb15392ca289cf1c0f9b0d9ecaf2ca82bff7fdccb44e4876870efa3dfde78eb900b6e2229da93433b326b700


In [7]:
alice = Wallet(seed_hex='7ef6f93701e5ca282c5680f2d39629afafaa00bf1a1cb8bf5a0f20d19c75c39b', display=True)
bob = Wallet(seed_hex='1d08199699f5d877efee7448b3c9eb7bac49a5689b28c92ecaf7ddebca375ae3')
carol = Wallet(seed_hex='609b1379370495999c9fe7fc9aa59cca39b26a2b274a1a97eef4bab24ef760e4')

Wallet seed: 7ef6f93701e5ca282c5680f2d39629afafaa00bf1a1cb8bf5a0f20d19c75c39b
Wallet secret exponent: 34858948105046340472739633959283104990204816252169379477184521129263669664787
Wallet private key: 4d1177272d648729dca17b4cc053f7c4a04b04a99b4fd8cc909bb95786175c13
Wallet public key: 8ec3a2d3a8b406879ad54e78c83aca27c2875972cd48a922938a1e8b31d00a6db7e3fd2dc35cddc91fdba2f6c2d19795a375a0041026bcfce49fb15ef7841815


In [8]:
class Account_Registry:
    """
    A class that represents a collection of accounts, public keys associated with a name, balance, and wallet.
    
    Attributes
    ----------
    accounts : dict
        dictionary of accounts; key is hex string representation of public_key; value is dictionary containing:
            name : string
                arbitrary pseudonym of wallet holder
            amount : int
                current balance
            wallet : Wallet
                wallet of account holder
    """
    def __init__(self, accounts=[]):
        """
        initializes accounts dict
        
        Parameters
        ----------
        accounts : list[ ( string, Wallet ) ], optional
            list of tuples containing the account pseudonym and the wallet object
        """
        self.accounts = dict()
        
        for account in accounts:
            pseudonym, wallet = account
            self.register_account(wallet, pseudonym)
    
    def register_account(self, wallet, pseudonym):
        """
        registers account by placing in accounts dict
        
        Parameters
        ----------
        wallet : Wallet
            wallet of account holder
        pseudonym : string
            arbitrary name of account holder
        """
        if not isinstance(wallet, Wallet):
            raise Exception(f'wallet "{wallet}" is not of type "Wallet"')
        
        public_key = wallet.public_key.to_string().hex()
        if not public_key in self.accounts.keys():
            self.accounts[public_key] = {
                'name': pseudonym,
                'amount': 0,
                'wallet': wallet
            }
        else:
            raise Exception(f'Account {public_key} already registered')
            
    def get_account(self, public_key):
        """
        retrieve account from public key of account holder
        
        Parameters
        ----------
        public_key : ecdsa.keys.VerifyingKey
            public key of account holder
        """
        public_key = public_key.to_string().hex()
        if public_key in self.accounts.keys():
            return self.accounts[public_key]
        else:
            raise Exception(f'Account {public_key} is not registered')
    
    def get_wallet(self, public_key):
        """
        retrieve wallet of account holder
        
        Parameters
        ----------
        public_key : ecdsa.keys.VerifyingKey
            public key of account holder
        """
        account = self.get_account(public_key)
        return account['wallet']
            
    def get_amount(self, public_key):
        """
        retrieve current balance of account holder
        
        Parameters
        ----------
        public_key : ecdsa.keys.VerifyingKey
            public key of account holder
        """
        account = self.get_account(public_key)
        return account['amount']
            
    def get_name(self, public_key):
        """
        retrieve psuedonym of account holder
        
        Parameters
        ----------
        public_key : ecdsa.keys.VerifyingKey
            public key of account holder
        """
        account = self.get_account(public_key)
        return account['name']

In [9]:
def create_timestamp(days_since_genesis=0,
                     hours_since_genesis=0,
                     minutes_since_genesis=0,
                     seconds_since_genesis=0,
                     display=False):
    """
    create a timestamp (unix time: seconds since epoch) by specifying time since genesis
    genesis is defined to be Monday, January 25, 2021, midnight, UTC
    
    Parameters
    ----------
    days_since_genesis : int, optional
        number of days since genesis (default 0)
    hours_since_genesis : int, optional
        number of hours since genesis (default 0)
    minutes_since_genesis : int, optional
        number of minutes since genesis (default 0)
    seconds_since_genesis : int, optional
        number of seconds since genesis (default 0)
    """

    # genesis time set to Monday, January 25, 2021, midnight of the first day of class, UTC
    genesis_time_struct = time.struct_time((2021, 1, 25, 0, 0, 0, 0, 25, 0))
    genesis_timestamp = calendar.timegm(genesis_time_struct)
    genesis_time_ascii = time.asctime(genesis_time_struct)

    seconds_since_genesis += minutes_since_genesis * 60
    seconds_since_genesis += hours_since_genesis * 60 * 60
    seconds_since_genesis += days_since_genesis * 60 * 60 * 24

    new_timestamp = genesis_timestamp + seconds_since_genesis
    new_time_struct = time.gmtime(new_timestamp)
    new_time_ascii = time.asctime(new_time_struct)

    if display:
        print(f'genesis time: {genesis_time_ascii}, timestamp: {genesis_timestamp}')
        if genesis_timestamp != new_timestamp:
            print(f'new time: {new_time_ascii}, timestamp: {new_timestamp}')

    return new_timestamp

In [10]:
class Hashable:
    """
    A class that represents a hashable object. Meant to be subclassed. Hashes with sha256.
    
    Attributes
    ----------
    hash_val : length 64 string encoding of hexadecimal number
        a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
    timestamp : int
        timestamp of object creation in Unix time (seconds since epoch)
    obj_type
        type of subclass extending Hashable
    """
    def __init__(self, timestamp=None):
        """
        initializes the hashable object
        
        hash_val is set to an empty string
        obj_type is set to name of class extending Hashable
        
        Parameters
        ----------
        timestamp : int, optional
            Unix time (seconds since epoch) representing time when object was created (default is current time)
        """
        self.hash_val = ''
        self.timestamp = self.set_timestamp(timestamp)
        self.obj_type = self.__class__.__name__
        
    def set_timestamp(self, timestamp):
        """
        sets the timestamp attribute
        
        Parameters
        ----------
        timestamp: int
            Unix time (seconds since epoch) representing time when object was created (if None, current time is used)
        """
        if timestamp is not None:
            return timestamp
        else:
            return time.time()
    
    def hash_obj(self, *args):
        """
        calculate hash of given arguments. arguments are byte encoded, concatenated, and hashed with SHA256.
        
        Parameters
        ----------
        *args : str
            an arbitrary number of string arguments
        """
        m = sha256()
            
        for a in args:
            if type(a) is bytes:
                m.update(a)
            elif type(a) is str:
                m.update(a.encode())
            else:
                raise Exception(f'arg {a} is wrong type: {type(a)}')
            
        return m.hexdigest()
        
    def display_creation(self, display):
        if display:
            time_struct = time.gmtime(self.timestamp)
            time_ascii = time.asctime(time_struct)
            
            print(f'{self.obj_type} creation time: {time_ascii}')
            print(f'{self.obj_type} hash: {self.hash_val}')

In [11]:
class Transaction(Hashable):
    """
    A class that represents a transaction between sender and recipient. Subclass of Hashable.
    
    Attributes
    ----------
    sender : ecdsa.keys.VerifyingKey
        public key of the transaction's sender
    recipient : ecdsa.keys.VerifyingKey
        public key of the transaction's recipient
    amount : int
        amount being transacted
    timestamp : int
        Unix time (seconds since epoch) of transaction creation
    hash_val : length 64 string encoding of hexadecimal number
        a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
    signature : bytes
        byte encoded digital signature, signed by private_key (via Wallet.sign_message()).
        message used for digital signature should be the transaction hash.
    """
    
    def __init__(self, sender, recipient, amount, timestamp=None, display=False):
        """
        initializes transaction with given parameters and hashes the transaction data (see Hashable.hash_transaction()).
        hash is concatenation of sender, recipient, amount and timestamp.
        
        Parameters
        ----------
        sender : ecdsa.keys.VerifyingKey
            public key of the transaction's sender
        recipient : ecdsa.keys.VerifyingKey
            public key of the transaction's recipient
        amount : int
            amount being transacted
        timestamp : int
            Unix time (seconds since epoch) of transaction
        """
        super().__init__(timestamp)
        self.sender = sender
        self.recipient = recipient
        self.amount = amount
        self.hash_val = self.hash_transaction()
        self.display_creation(display)
        
    def hash_transaction(self):
        """
        hash the transaction details (see Hashable.hash_obj())
        """
        return self.hash_obj(
            self.sender.to_string(),
            self.recipient.to_string(),
            str(self.amount),
            str(self.timestamp))
    
    def add_signature(self, signature):
        """
        set the digital signature of the transaction hash
        
        Parameters
        ----------
        signature : bytes
            byte encoded digital signature retrieved from private key of sender (see Wallet.sign_message())
        """
        self.signature = signature
    
    def display_creation(self, display):
        if display:
            super().display_creation(display)
            print(f'{self.obj_type} sender: {self.sender.to_string().hex()}')
            print(f'{self.obj_type} recipient: {self.recipient.to_string().hex()}')
            print(f'{self.obj_type} amount: {self.amount}')

In [12]:
class Block(Hashable):
    """
    A class that represents a Block. Subclass of Hashable.
    A Block includes collection of transactions (including coinbase).
    Most times a Block is mined, but sometimes a shell Block is created where no mining occurs for verification.
    
    Attributes
    ----------
    previous_hash : length 64 string encoding of hexadecimal number (see Hashable.hash_val)
        a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
        represents hash_val of previous block
    coinbase : Transaction
        a transaction where the recipient is set to the miner of the block (rewarded with mining fee, see Blockchain.mining_reward)
    transactions : list[ Transaction ], may be empty list
        a list of transactions to be included in the block; should be retrieved from transaction pool (see Blockchain.get_n_transactions())
    transactions_hash : length 64 string encoding of hexadecimal number (see Hashable.hash_val)
        a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
        represents hash derived from concatenation of all individual transaction hashes
    target : length 64 string encoding of hexadecimal number (see Blockchain.target)
        a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
        Block.hash_val must be less than or equal to this number, which is achieved in mining process
    timestamp : int
        Unix time (seconds since epoch) of block creation
    nonce : int
        value to iterate in order to generate random hash_vals
        nonce derived from mining processs and has property such that concatenation with Block data produces hash less than or equal to target
    hash_val : length 64 string encoding of hexadecimal number (see Hashable.hash_val)
        a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
    mine_time : int
        number of seconds it took to find ideal nonce value
    """

    def __init__(self,
                 previous_hash,
                 coinbase_transaction,
                 transactions,
                 target,
                 timestamp=None,
                 mine=True,
                 display=False):
        """
        initializes block with data, and then starts mining (if specified) to find ideal nonce
        
        Parameters
        ----------
        previous_hash : length 64 string encoding of hexadecimal number (see Hashable.hash_val)
            a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
        coinbase_transaction : Transaction
            a transaction where the recipient is set to the miner of the block (rewarded with mining fee, see Blockchain.mining_reward)
        transactions : list[ Transaction ], can be empty list
            a list of transactions to be included in the block; should be retrieved from transaction pool (see Blockchain.get_n_transactions())
        target : length 64 string encoding of hexadecimal number (see Blockchain.target)
            a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
            Block.hash_val must be less than or equal to this number, which is achieved in mining process
        timestamp : int, optional
            Unix time (seconds since epoch) of block creation (default is current time)
        mine : bool
            specifies whether the mining process should be initialized (default True)
            mine=False useful for creating shell block in order to use in verification process (see Blockchain.verify_block())
        """
        super().__init__(timestamp)
        self.previous_hash = previous_hash
        self.coinbase = coinbase_transaction
        self.transactions = transactions
        self.transactions_hash = self.hash_transactions(coinbase_transaction, transactions)
        self.target = target
        self.nonce = 0
        self.hash_val = self.hash_block()
        self.mine_block(mine=mine)
        self.display_creation(display)
    
    def hash_transactions(self, coinbase_transaction, transactions):
        """
        calculate hash of all transactions by hashing the concatenation of each individual transaction hash
        
        Parameters
        ----------
        coinbase : Transaction
            a transaction where the recipient is set to the miner of the block (rewarded with mining fee, see Blockchain.mining_reward)
        transactions : list[ Transaction ], may be empty list
            a list of transactions to be included in the block; should be retrieved from transaction pool (see Blockchain.get_n_transactions())
        """
        m = sha256()

        for tx in [coinbase_transaction]+transactions:
            if not isinstance(tx, Transaction):
                raise Exception(f'tx "{tx}" is not of type "Transaction"')
                
            m.update(tx.hash_val.encode())
            
        return m.hexdigest()
  
    def hash_block(self):
        """
        calculate hash of the block by hashing concatenation of nonce, the previous block hash, block creation timestamp, and hash of transactions
        """
        return self.hash_obj(str(self.nonce),
                             str(self.previous_hash),
                             str(self.timestamp),
                             str(self.transactions_hash))
    
    def mine_block(self, mine=True):
        """
        mine the block by iterating nonce and hashing block. once block hash less than or equal to target, stop and log time it took in seconds
        
        Parameters
        ----------
        mine : bool
            specifies whether mining should occur
        """
        if mine:
            start = time.time()
            while int(self.hash_val, 16) > int(self.target, 16):
                self.nonce += 1
                self.hash_val = self.hash_block()
            end = time.time()
            self.mine_time = end - start
    
    def display_creation(self, display):
        if display:
            super().display_creation(display)
            print(f'{self.obj_type} miner: {self.coinbase.sender.to_string().hex()}')
            print(f'{self.obj_type} reward: {self.coinbase.amount}')
            print(f'{self.obj_type} nonce: {self.nonce}')
            print(f'{self.obj_type} previous hash: {self.previous_hash}')
            print(f'{self.obj_type} mine time: {self.mine_time:.6f} seconds')

In [13]:
class Blockchain:
    """
    A class that represents a chain of blocks, and associated methods to modify or query that chain.
    Contains transaction pool, and associated methods to retrieve transactions from pool.
    Contains Blockchain parameters such as reward and mining difficulty (target).
    Contains registry of accounts.
    Contains methods to verify additions to blockchain.
    
    Attributes
    ----------
    chain : list[ Block ]
        a list of Blocks in order of when they were mined (increasing value of block timestamps)
    transaction_pool : list[ Transaction ]
        a list of transactions waiting to be included in blocks
    account_registry : Account_Registry
        a registry of accounts associated with the blockchain, public keys associated with a name, balance, and wallet
    target : length 64 string encoding of hexadecimal number
        a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
        Block.hash_val must be less than or equal to this number, which is achieved in mining process (see Blockchain.verify_block())
    mining_reward : int
        reward for successfully mining a block and including in blockchain (default is 100)
    """
    
    def __init__(self, account_registry=None, num_zero_difficulty=1, reward_amount=100, display=False):
        """
        initializes blockchain by creating empty chain and transaction pool, and account registry, as well as blockchain settings.
        
        Parameters
        ----------
        account_registry : Account_Registry
            public keys associated with a name, balance, and wallet (default is None, where empty account registry is created)
        num_zero_difficulty : int
            difficulty setting for mining process, specified by number of leading 0's, used in determining target.
            all blocks added to chain must have hash values with as many leading 0's.
            must be between 1 and 64.
            default is None, in which case 1 leading 0 is used, followed by 63 F values.
        reward_amount : int
            amount to reward a miner of a block
        """
        
        self.chain = []
        self.transaction_pool = []
        
        if account_registry and isinstance(account_registry, Account_Registry):
            self.account_registry = account_registry
        else:
            self.account_registry = Account_Registry()
        
        if num_zero_difficulty > 1 and num_zero_difficulty <= 64:
            self.target = '0'*num_zero_difficulty + 'F'*(64 - num_zero_difficulty)
        else:
            raise Exception(f'num_zero_difficulty of {num_zero_difficulty} is invalid, must be int > 1 and <= 64')
            
        if reward_amount >= 0:
            self.mining_reward = reward_amount
        else:
            raise Exception(f'reward amount {reward_amount} invalid, must be int > 0')
        
    def get_target(self):
        """
        retrieve target (difficulty). hash of a block must be less than this value in order to be included in blockchain.
        """
        return self.target
        
    def get_last_hash(self):
        """
        get the hash of the most recently verified block added to blockchain.
        if no blocks have been added yet, hexstring of all '0' values is returned.
        """
        if len(self.chain) == 0:
            return '0'*64
        head_block = self.chain[-1]
        return head_block.hash_val
    
    def verify_transaction(self, transaction, display=False):
        """
        verify a transaction.
        for a transaction to be valid:
            its signature attribute must be set (via Transaction.add_signature())
            it must be set to the digital signature (via Wallet.sign_message()) of the transaction hash (Transaction.hash_val)
            the transaction sender (ecdsa.keys.VerifyingKey) must be able to verify the signature (ecdsa.keys.VerifyingKey.verify())
            
        Parameters
        ----------
        transaction : Transaction
            the transaction to verify, with the signature attribute set (via Transaction.add_signature())
        """
        if not isinstance(transaction, Transaction):
            raise Exception(f'tx "{transaction}" is not of type "Transaction"')
            
        if not isinstance(transaction.sender, VerifyingKey):
            raise Exception(f'tx sender "{transaction.sender}" not of type "ecdsa.VerifyingKey"')

        if not isinstance(transaction.recipient, VerifyingKey):
            raise Exception(f'tx recipient "{transaction.recipient}" not of type "ecdsa.VerifyingKey"')
            
        if transaction.sender.to_string().hex() not in self.account_registry.accounts:
            raise Exception(f'tx sender {transaction.sender.to_string().hex()} not registered')
            
        if transaction.recipient.to_string().hex() not in self.account_registry.accounts:
            raise Exception(f'tx recipient {transaction.recipient.to_string().hex()} not registered')
            
        if not transaction.sender.verify(transaction.signature, transaction.hash_val.encode()):
            raise Exception(f'{transaction.sender.to_string().hex()} has not verified this transaction')
            
    def add_transaction(self, transaction, display=False):
        """
        verify a transaction (Blockhain.verify_transaction()) and add it to the transaction pool
        """
        self.verify_transaction(transaction)    
        
        self.transaction_pool.append(transaction)
        
        if display:
            sender_name = self.account_registry.accounts[transaction.sender.to_string().hex()]['name']
            recipient_name = self.account_registry.accounts[transaction.recipient.to_string().hex()]['name']
            print(f'tx sender: {sender_name} ({transaction.sender.to_string().hex()})')
            print(f'tx recipient: {recipient_name} ({transaction.recipient.to_string().hex()})')
            print(f'tx amount: {transaction.amount}')
        
    def add_transactions(self, transactions, display=False):
        """
        verify (Blockhain.verify_transaction()) and add (Blockchain.add_transaction()) a list of transactions to the transaction pool
        """
        for tx in transactions:
            self.add_transaction(tx, display)
        
    def get_n_transactions(self, num):
        """
        remove (up to) specified number of transactions from the transaction pool and return as a list.
        elements are removed and returned in FIFO order.
        
        Parameters
        ----------
        num : int
            the number of transactions to retrieve from transaction pool
            if num > len(transaction_pool), then len(transaction_pool) number of transactions returned
        """
        n_transactions = self.transaction_pool[:num]
        self.transaction_pool = self.transaction_pool[num:]
        
        return n_transactions
    
    def verify_block(self, block):
        """
        verify a given block.
        for a block to be valid:
            the coinbase must be less than or equal to the mining reward (Blockchain.mining_reward)
            the attributes of the provided block (including its nonce value) should produce a valid hash
                this is verified by constructing a shell Block (mine=False), hashing it, and comparing the hash against the provided block
            the hash of the provided block must be less than the target (Blockchain.target)
            the individual transactions included in the block must be valid (Blockchain.verify_transaction())
            
        Parameters
        ----------
        block : Block
            a valid block
        """
        if not isinstance(block, Block):
            raise Exception(f'block "{block}" is not of type "Block"')
            
        if block.coinbase.amount > self.mining_reward:
            raise Exception(f'{block.coinbase.amount} exceeds mining reward of {self.mining_reward}')

        verification_block = Block(previous_hash=self.get_last_hash(),
                                   coinbase_transaction=block.coinbase,
                                   transactions=block.transactions,
                                   target=self.target,
                                   timestamp=block.timestamp,
                                   mine=False)
        verification_block.nonce = block.nonce
        verification_block.hash_val = verification_block.hash_block()
        
        if not verification_block.hash_val == block.hash_val:
            raise Exception(f'block {block.hash_val} has an invalid hash val {block.hash_val}')
            
        if not int(verification_block.hash_val, 16) <= int(self.target, 16):
            raise Exception(f'block {block.hash_val} is not below target of {self.target}')
        
        for transaction in [verification_block.coinbase] + verification_block.transactions:
            self.verify_transaction(transaction)
    
    def add_block(self, block):
        """
        verify (Blockchain.verify_block()) and add a block to the chain
        
        Parameters
        ----------
        block : Block
            a valid block
        """
        self.verify_block(block)
        self.chain.append(block)

    def verify_chain(self, chain):
        """
        verify a chain of blocks, where each block must be valid (Blockchain.verify_block())
        
        Parameters
        ----------
        chain : list[ Block ]
            a list of valid blocks in order of creation (by timestamp)
        """
        for block in chain:
            self.verify_block(block)
            
    def find_block_idx(self, block_hash):
        """
        find the index of a block matching the provided block hash
        
        Parameters
        ----------
        block_hash : length 64 string encoding of hexadecimal number (see Hashable.hash_val)
            a 64 digit hexadecimal number in string format (i.e. 32 bytes or 256 bits)
        """
        for i, block in enumerate(chain):
            if block.hash_val == block_hash:
                return i
        raise Exception(f'could not find block with hash value {append_block.hash_val}')
    
    def add_subchain(self, subchain):
        """
        overwrite the current blockchain by attaching a subchain (if valid).
        the first block in the subchain (oldest timestamp) is queried for the hash of the previous block (Block.prev_hash),
        the subchain is then attached to the specified previous block, overwriting the current chain of blocks.
        the new chain formed by attaching the subchain is considered valid if:
            all blocks in the new chain must be valid (Blockhain.verify_chain())
            the size of the new chain must be greater than the size of the current chain
            
        Paramters
        ---------
        subchain : List[ Block ]
            list of blocks where the first block's prev_hash attribute is set to some block that is already in the current chain
        """
        append_block = subchain[0]
        append_idx = find_block_idx(append_block.prev_hash) + 1    
        
        prefix_chain = self.chain[0:append_idx]
        new_chain = prefix_chain + subchain
        
        curr_chain_size = len(self.chain)
        new_chain_size = len(new_chain)
        if new_chain_size <= curr_chain_size:
            raise Exception(f'proposed chain of size {new_chain_size} is less than current blockchain of size {curr_chain_size}')
        
        self.verify_chain(new_chain)
        
        self.chain = new_chain
            
    def display_blockchain(self):
        """
        display all blocks and the transactions they contain
        """
        for i, block in enumerate(self.chain):
            print(f'Block {i}')

            block_time_struct = time.gmtime(block.timestamp)
            block_time_ascii = time.asctime(block_time_struct)
            
            print(f'Block {i} creation time: {block_time_ascii}')
            print(f'Block {i} hash: {block.hash_val}')
            miner = block.coinbase.sender.to_string().hex()
            miner_name = self.account_registry.accounts[miner]['name']
            print(f'Block {i} miner: {miner_name} ({miner})')
            print(f'Block {i} reward: {block.coinbase.amount}')
            print(f'Block {i} nonce: {block.nonce}')
            print(f'Block {i} previous hash: {block.previous_hash}')
            print(f'Block {i} mine time: {block.mine_time:.6f} seconds')
            
            
            for ii, tx in enumerate(block.transactions):
                print(f'Block {i} - Transaction {ii}')
                
                tx_time_struct = time.gmtime(tx.timestamp)
                tx_time_ascii = time.asctime(tx_time_struct)
                
                print(f'Block {i} - Transaction {ii} creation time: {tx_time_ascii}')
                print(f'Block {i} - Transaction {ii} hash: {tx.hash_val}')
                sender = tx.sender.to_string().hex()
                sender_name = self.account_registry.accounts[sender]['name']
                print(f'Block {i} - Transaction {ii} sender: {sender_name} ({sender})')
                recipient = tx.recipient.to_string().hex()
                recipient_name = self.account_registry.accounts[recipient]['name']
                print(f'Block {i} - Transaction {ii} recipient: {recipient_name} ({recipient})')
                print(f'Block {i} - Transaction {ii} amount: {tx.amount}')

In [14]:
def sign_transaction(wallet, transaction):
    """
    a transaction's signature attribute must be set before including in a block on the blockchain.
    this function produces a digital signature of the given transaction with the private key contained in the given wallet.
    the message used is the transaction hash.
    
    Parameters
    ----------
    wallet : Wallet
        wallet containing the private key (ecdsa.keys.SigningKey) with which to sign (Wallet.sign_message()) the transaction
    transaction : Transaction
        the transaction to sign and set (Transaction.add_signature())
    """
    if not isinstance(wallet, Wallet):
        raise Exception(f'wallet "{wallet}" not of type "Wallet"')
        
    if not isinstance(transaction, Transaction):
        raise Exception(f'tx "{transaction}" not of type "Transaction"')
        
    transaction.add_signature(wallet.sign_message(transaction.hash_val))

In [15]:
account_registry = Account_Registry( accounts=[('alice', alice), ('bob', bob), ('carol', carol)] )

In [16]:
blockchain = Blockchain(account_registry=account_registry, num_zero_difficulty=2, display=True)

In [17]:
genesis_timestamp = create_timestamp(0,0,0,0, display=True)

nakamoto = Wallet(seed_hex='6daa908eb3d309183a9f4789a00f51b18542da1417be0cdd799081228236dae9', display=True)
blockchain.account_registry.register_account(nakamoto, 'nakamoto')
genesis_coinbase_transaction = Transaction(nakamoto.public_key, nakamoto.public_key, blockchain.mining_reward, genesis_timestamp, display=True)
sign_transaction(wallet=nakamoto, transaction=genesis_coinbase_transaction)

genesis_block = Block(previous_hash=blockchain.get_last_hash(),
                      coinbase_transaction=genesis_coinbase_transaction,
                      transactions=[],
                      target=blockchain.target,
                      timestamp=genesis_timestamp,
                      mine=True,
                      display=display)

blockchain.add_block(genesis_block)

genesis time: Mon Jan 25 00:00:00 2021, timestamp: 1611532800
Wallet seed: 6daa908eb3d309183a9f4789a00f51b18542da1417be0cdd799081228236dae9
Wallet secret exponent: 85314299752260761361628249051057399690231672481648748497211137228769388769381
Wallet private key: bc9e2eb5d3a6857f52803f57750a2e7c9a75e8d1a14784e61f38845ecc6db065
Wallet public key: 57b28b2862c10eb35a0cc9799543ad75ed88c1151396993e5d43bb2f1851acd2665e92bff0daf899374c1bf6723b84d112bdfed23ace5fa2f00e629080f7d638
Transaction creation time: Mon Jan 25 00:00:00 2021
Transaction hash: 61cee37ebe97ea40aab267dcd1d14ce9da3098f314b1bcb826534945a837e194
Transaction sender: 57b28b2862c10eb35a0cc9799543ad75ed88c1151396993e5d43bb2f1851acd2665e92bff0daf899374c1bf6723b84d112bdfed23ace5fa2f00e629080f7d638
Transaction recipient: 57b28b2862c10eb35a0cc9799543ad75ed88c1151396993e5d43bb2f1851acd2665e92bff0daf899374c1bf6723b84d112bdfed23ace5fa2f00e629080f7d638
Transaction amount: 100
Block creation time: Mon Jan 25 00:00:00 2021
Block hash: 0087653

In [18]:
tx1_ts = create_timestamp(7,0,0,0)
tx1 = Transaction(alice.public_key, bob.public_key, 50, tx1_ts, display=True)
sign_transaction(wallet=alice, transaction=tx1)

tx2_ts = create_timestamp(7,0,0,1)
tx2 = Transaction(alice.public_key, carol.public_key, 10, tx2_ts, display=True)
sign_transaction(wallet=alice, transaction=tx2)

Transaction creation time: Mon Feb  1 00:00:00 2021
Transaction hash: 4f7963fcb8c88541c42f3e10964072f4b168569cc2ad71362c350a90fdea8d88
Transaction sender: 8ec3a2d3a8b406879ad54e78c83aca27c2875972cd48a922938a1e8b31d00a6db7e3fd2dc35cddc91fdba2f6c2d19795a375a0041026bcfce49fb15ef7841815
Transaction recipient: 0010b324b02beba152fd0f285e153f0cb153166e02fc32c5826f8fae8d6dbd2d54d6c5645ec2d14c29a20d2477f2bbe2b73b086fabec8d27bf98350ed8cf2163
Transaction amount: 50
Transaction creation time: Mon Feb  1 00:00:01 2021
Transaction hash: 590196db1bb78a3aa5f791aa405b86f74544e18b91b8bd7cc752d90fdf540a01
Transaction sender: 8ec3a2d3a8b406879ad54e78c83aca27c2875972cd48a922938a1e8b31d00a6db7e3fd2dc35cddc91fdba2f6c2d19795a375a0041026bcfce49fb15ef7841815
Transaction recipient: 4ccb71339d78b7bb04417f04c7dc50d47b30d04568105e69e69490dd06d223793e20a090a9cbcd1743ac7b5abfe0e76cea4cbd105e331447232a32c9cd6a4838
Transaction amount: 10


In [19]:
blockchain.add_transactions([tx1, tx2], display=True)

tx sender: alice (8ec3a2d3a8b406879ad54e78c83aca27c2875972cd48a922938a1e8b31d00a6db7e3fd2dc35cddc91fdba2f6c2d19795a375a0041026bcfce49fb15ef7841815)
tx recipient: bob (0010b324b02beba152fd0f285e153f0cb153166e02fc32c5826f8fae8d6dbd2d54d6c5645ec2d14c29a20d2477f2bbe2b73b086fabec8d27bf98350ed8cf2163)
tx amount: 50
tx sender: alice (8ec3a2d3a8b406879ad54e78c83aca27c2875972cd48a922938a1e8b31d00a6db7e3fd2dc35cddc91fdba2f6c2d19795a375a0041026bcfce49fb15ef7841815)
tx recipient: carol (4ccb71339d78b7bb04417f04c7dc50d47b30d04568105e69e69490dd06d223793e20a090a9cbcd1743ac7b5abfe0e76cea4cbd105e331447232a32c9cd6a4838)
tx amount: 10


In [20]:
b1_txs = blockchain.get_n_transactions(2)

In [21]:
genesis_block_hash = blockchain.get_last_hash()

In [22]:
b1_ts = create_timestamp(7,0,1,0)
b1_coinbase_tx = Transaction(alice.public_key, alice.public_key, 100, b1_ts, display=True)
sign_transaction(wallet=alice, transaction=b1_coinbase_tx)

block1 = Block(
    genesis_block_hash,
    b1_coinbase_tx,
    b1_txs,
    blockchain.target,
    timestamp=b1_ts,
    mine=True,
    display=True)

Transaction creation time: Mon Feb  1 00:01:00 2021
Transaction hash: 64ae7e1bfd911b1be0cfdc575d132684fb2b5787360ff6e6d1efe17f99d97c8d
Transaction sender: 8ec3a2d3a8b406879ad54e78c83aca27c2875972cd48a922938a1e8b31d00a6db7e3fd2dc35cddc91fdba2f6c2d19795a375a0041026bcfce49fb15ef7841815
Transaction recipient: 8ec3a2d3a8b406879ad54e78c83aca27c2875972cd48a922938a1e8b31d00a6db7e3fd2dc35cddc91fdba2f6c2d19795a375a0041026bcfce49fb15ef7841815
Transaction amount: 100
Block creation time: Mon Feb  1 00:01:00 2021
Block hash: 009af4a6ca2aeb948531dbac1ea140510a08b22d45e726af7838a1d8f43957c5
Block miner: 8ec3a2d3a8b406879ad54e78c83aca27c2875972cd48a922938a1e8b31d00a6db7e3fd2dc35cddc91fdba2f6c2d19795a375a0041026bcfce49fb15ef7841815
Block reward: 100
Block nonce: 613
Block previous hash: 00876534b9fb3d471dc0bf05ad2742dda3eacab68555cec37d72a0822ff3958a
Block mine time: 0.002965 seconds


In [23]:
blockchain.add_block(block1)

In [24]:
blockchain.display_blockchain()

Block 0
Block 0 creation time: Mon Jan 25 00:00:00 2021
Block 0 hash: 00876534b9fb3d471dc0bf05ad2742dda3eacab68555cec37d72a0822ff3958a
Block 0 miner: nakamoto (57b28b2862c10eb35a0cc9799543ad75ed88c1151396993e5d43bb2f1851acd2665e92bff0daf899374c1bf6723b84d112bdfed23ace5fa2f00e629080f7d638)
Block 0 reward: 100
Block 0 nonce: 810
Block 0 previous hash: 0000000000000000000000000000000000000000000000000000000000000000
Block 0 mine time: 0.003881 seconds
Block 1
Block 1 creation time: Mon Feb  1 00:01:00 2021
Block 1 hash: 009af4a6ca2aeb948531dbac1ea140510a08b22d45e726af7838a1d8f43957c5
Block 1 miner: alice (8ec3a2d3a8b406879ad54e78c83aca27c2875972cd48a922938a1e8b31d00a6db7e3fd2dc35cddc91fdba2f6c2d19795a375a0041026bcfce49fb15ef7841815)
Block 1 reward: 100
Block 1 nonce: 613
Block 1 previous hash: 00876534b9fb3d471dc0bf05ad2742dda3eacab68555cec37d72a0822ff3958a
Block 1 mine time: 0.002965 seconds
Block 1 - Transaction 0
Block 1 - Transaction 0 creation time: Mon Feb  1 00:00:00 2021
Block 1 -